In [1]:
%load_ext autoreload
%autoreload 2

In [33]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from models_ext_ms import SynthesizerTrn as SynthesizerTrnExt
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    # text_norm = text_to_sequence(text, hps.data.text_cleaners)
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## VCTK

In [34]:
hps = utils.get_hparams_from_file("./logs/vctk_16k/config.json")

In [35]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).to(device)
_ = net_g.eval()

_ = utils.load_checkpoint("logs/vctk_16k/G_500000.pth", net_g, None)

In [5]:
speaker_id = 5

stn_tst = get_text("institutional investors and hedge funds own zero point nine zero of the companys stock.", hps)

with torch.no_grad():
    x_tst = stn_tst.to(device).unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
    sid = torch.LongTensor([speaker_id]).to(device)
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
    
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [6]:
# write("./configs/a.wav", 16000, audio)

### AfroTTS

In [36]:
hps = utils.get_hparams_from_file("logs/afrotts/config.json")

In [37]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).to(device)
_ = net_g.eval()

_ = utils.load_checkpoint("logs/afrotts/G_300000.pth", net_g, None)

In [ ]:
# import pandas as pd

# train = pd.read_csv("data/afritts-train-clean.csv")

In [ ]:
# train.head()

In [ ]:
# train[train.accent == 'Siswati'].user_ids_num.unique()

In [54]:
speaker_id = 2

stn_tst = get_text("institutional investors and hedge funds own zero point nine zero of the companys stock.", hps)

with torch.no_grad():
    x_tst = stn_tst.to(device).unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
    sid = torch.LongTensor([speaker_id]).to(device)
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, 
                        noise_scale=.667, noise_scale_w=0.8, 
                        length_scale=1, eval_mode=True)
    audio = audio[0][0,0].data.cpu().float().numpy()
    
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [49]:
# from scipy.io.wavfile import write
# dst_path = "/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/samples/sampl_male3.wav"
# write(dst_path, 16000, audio)

### VITS with external speaker embedding

In [20]:
# from data_utils_ext_ms import TextAudioLoader as , TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models_ext_ms import SynthesizerTrn as SynthesizerTrnExt

In [21]:
hps = utils.get_hparams_from_file("logs/afrotts_ft_ext2/config.json")

In [22]:
import json
import pickle as pkl
def load_speaker_emb(path):
    print(f"Loading external speaker embeddings from {path}")
    if path.split(".")[-1] == "pkl":
        with open(path, "rb") as read_file:
            speaker_embeddings_dict = pkl.load(read_file)
    elif path.split(".")[-1] == "json":
        with open(path, "r") as read_file:
            speaker_embeddings_dict = json.load(read_file)
    else:
        raise TypeError("Speaker embedding type unrecognized")
    return speaker_embeddings_dict

In [21]:
# speaker_emb_dict = load_speaker_emb("/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/embeddings/afritts_emb_intsp.pkl")


In [23]:
speaker_emb_dict = load_speaker_emb(hps.data.speaker_emb_path)

Loading external speaker embeddings from /srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/embeddings/afritts_emb_intsp.pkl


In [24]:
device="cpu"
net_g = SynthesizerTrnExt(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).to(device)
_ = net_g.eval()

_ = utils.load_checkpoint("logs/afrotts_ft_ext2/G_340000.pth", net_g, None)

In [25]:
speaker_emb = speaker_emb_dict.get("6c74fccba13870402ac15ea14f2ad70c_Rm9hVVH6")

In [26]:
speaker_emb.shape

(256,)

In [39]:
stn_tst = get_text("institutional investors and hedge funds own zero point nine zero of the companys stock.", hps)

with torch.no_grad():
    x_tst = stn_tst.to(device).unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
    sid = torch.tensor(speaker_emb).reshape(1, -1).to(device)
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, 
                        noise_scale=.667, noise_scale_w=0.8, 
                        length_scale=1, eval_mode=True,)[0][0,0].data.cpu().float().numpy()
    
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

### Speaker Interpolation

In [67]:
hps = utils.get_hparams_from_file("logs/afrotts_vctk_ft/config.json")

In [68]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).to(device)
_ = net_g.eval()

_ = utils.load_checkpoint("logs/afrotts_vctk_ft/G_112000.pth", net_g, None)

AssertionError: 

In [28]:
speaker_ids = [159, 162] # speaker ids to interpolate

stn_tst = get_text("Alh. hassan was here to greet him.", hps)

with torch.no_grad():
    x_tst = stn_tst.to(device).unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
    sid = torch.LongTensor([speaker_ids]).to(device)
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
    
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

### Voice Conversion

In [ ]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))